In [ ]:
!pip install git+https://github.com/aimclub/eXplain-NNs &> /dev/null

# !wget https://raw.githubusercontent.com/aimclub/eXplain-NNs/main/requirements.txt
# !pip install -r requirements.txt
! pip install torchmetrics &> /dev/null
! pip install pip install giotto-ph==0.2.2 &> /dev/null
! pip install pip install giotto-tda==0.6.0 &> /dev/null
! pip install umap-learn==0.5.3 &> /dev/null


In [ ]:
from torchvision.datasets import MNIST
import torch
import torch.nn as nn
import torchvision.transforms as TF
from tqdm.auto import tqdm

In [ ]:
from eXNN.bayes import DropoutBayesianWrapper

In [ ]:
train_ds = MNIST(root='./.cache', train=True, download=True, transform=TF.ToTensor())
test_ds = MNIST(root='./.cache', train=False, download=False, transform=TF.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 29503289.65it/s]


Extracting ./.cache/MNIST/raw/train-images-idx3-ubyte.gz to ./.cache/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 117379548.28it/s]

Extracting ./.cache/MNIST/raw/train-labels-idx1-ubyte.gz to ./.cache/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 24279745.95it/s]


Extracting ./.cache/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.cache/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14883225.60it/s]

Extracting ./.cache/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.cache/MNIST/raw



In [ ]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=36, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=36, shuffle=False)

In [ ]:
num_classes = len(train_ds.classes)

In [ ]:
model = nn.Sequential(nn.Flatten() ,nn.Linear(28*28, 128),
                       nn.ReLU(), nn.Linear(128, 64),
                       nn.ReLU(), nn.Linear(64, num_classes), nn.Softmax(dim=1))
optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
criterion = nn.CrossEntropyLoss()
images, labels = next(iter(train_dl))
images = images.view(images.shape[0], -1)
logps = model(images)
loss = criterion(logps, labels)

In [ ]:
# train
n_epochs = 20
for e in range(n_epochs):
    running_loss = 0
    for images, labels in train_dl:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_dl)))

Epoch 0 - Training loss: 2.264180534960055
Epoch 1 - Training loss: 1.8589953394370946
Epoch 2 - Training loss: 1.6964962998763773
Epoch 3 - Training loss: 1.606024907889592
Epoch 4 - Training loss: 1.5694306460267853
Epoch 5 - Training loss: 1.558262092450742
Epoch 6 - Training loss: 1.550862583201591
Epoch 7 - Training loss: 1.54485444280296
Epoch 8 - Training loss: 1.539998533200846
Epoch 9 - Training loss: 1.5355852614209977
Epoch 10 - Training loss: 1.531968993416931
Epoch 11 - Training loss: 1.5285690525869589
Epoch 12 - Training loss: 1.5254286123594982
Epoch 13 - Training loss: 1.5223983976035755
Epoch 14 - Training loss: 1.5196804830823463
Epoch 15 - Training loss: 1.5170987689287705
Epoch 16 - Training loss: 1.5149314093175017
Epoch 17 - Training loss: 1.512628909493179
Epoch 18 - Training loss: 1.5106675631998538
Epoch 19 - Training loss: 1.5088391979082516


In [ ]:
correct_count, all_count = 0, 0
for images,labels in test_dl:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9538


In [ ]:
DropoutBayesianWrapper

In [ ]:
# build Dropout based bayesianized model
bayes_model = DropoutBayesianWrapper(model, 'basic', 0.1, None, None)

In [ ]:
# predict
n_iter = 5
prediction = bayes_model.predict(images, n_iter)

In [ ]:
prediction.keys()

dict_keys(['mean', 'std'])

In [ ]:
prediction['mean']

tensor([[2.0785e-13, 2.0895e-15, 3.3889e-12, 1.8083e-13, 9.9996e-01, 2.7775e-09,
         1.8968e-08, 2.3739e-07, 1.7986e-09, 4.4474e-05],
        [5.3205e-10, 4.9924e-18, 1.5969e-10, 8.9747e-12, 4.1226e-02, 1.2289e-07,
         1.1088e-13, 2.6331e-02, 3.3986e-05, 9.3241e-01],
        [3.8153e-12, 2.2360e-14, 8.6113e-11, 3.7582e-13, 9.9999e-01, 5.1919e-09,
         6.7374e-08, 3.3685e-07, 7.0977e-10, 5.4580e-06],
        [1.7025e-07, 1.7149e-08, 2.2754e-01, 7.6450e-01, 1.7627e-09, 1.1286e-04,
         6.3117e-11, 3.3387e-07, 7.8441e-03, 1.9481e-08],
        [2.2944e-06, 1.2958e-10, 1.4721e-04, 1.2801e-10, 1.9821e-03, 9.0356e-05,
         9.9778e-01, 7.2889e-08, 9.8936e-07, 2.0013e-07],
        [4.5497e-14, 1.3074e-11, 1.1793e-10, 4.0089e-12, 9.9990e-01, 3.6573e-06,
         5.2466e-08, 6.6548e-05, 6.6881e-08, 3.0657e-05],
        [6.2195e-14, 9.9587e-01, 1.8758e-04, 5.0828e-04, 5.5840e-06, 9.0976e-08,
         8.6983e-09, 3.4217e-03, 2.6080e-06, 1.5108e-06],
        [5.6196e-08, 1.0981

In [ ]:
prediction['std']

tensor([[2.8321e-13, 4.5394e-15, 4.1571e-12, 3.9780e-13, 6.7457e-05, 4.4869e-09,
         3.7222e-08, 3.7944e-07, 1.8420e-09, 6.7084e-05],
        [9.3369e-10, 6.8126e-18, 3.4164e-10, 6.5199e-12, 8.1769e-02, 1.3547e-07,
         2.1573e-13, 5.8168e-02, 5.1049e-05, 8.6575e-02],
        [7.0963e-12, 4.5108e-14, 1.5339e-10, 8.0990e-13, 8.8348e-06, 7.7907e-09,
         1.1782e-07, 5.5556e-07, 1.1267e-09, 8.2416e-06],
        [2.2098e-07, 3.6732e-08, 3.9250e-01, 3.9060e-01, 3.7658e-09, 2.3282e-04,
         1.2911e-10, 5.4734e-07, 1.6971e-02, 2.5597e-08],
        [4.1477e-06, 2.1063e-10, 2.9859e-04, 1.6368e-10, 3.9680e-03, 1.5986e-04,
         3.9006e-03, 1.0487e-07, 1.3276e-06, 3.8904e-07],
        [7.7002e-14, 2.7217e-11, 1.9129e-10, 8.9364e-12, 1.7908e-04, 6.9304e-06,
         9.2264e-08, 1.3850e-04, 1.3718e-07, 4.3947e-05],
        [1.3880e-13, 5.4619e-03, 2.8126e-04, 6.9102e-04, 1.0400e-05, 8.4543e-08,
         9.4659e-09, 4.6524e-03, 4.9639e-06, 1.9879e-06],
        [1.2561e-07, 2.4529